In [2]:
import sklearn.tree
% load_ext autoreload
% autoreload 2

UsageError: Line magic function `%` not found.


In [72]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import (
    plot_confusion_matrix,
    confusion_matrix,
    ConfusionMatrixDisplay,
)
import matplotlib.pyplot as plt
import os
import numpy as np
import re

from tqdm import tqdm
import spacy
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB as NaiveBayes
from sklearn.tree import DecisionTreeClassifier as DTC

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score,precision_score, recall_score
from sklearn.neural_network import MLPClassifier
from tools.read import get_data

In [3]:
class WordVectorTransformerPlus(TransformerMixin, BaseEstimator):
    def __init__(self, model="fr_core_news_lg"):
        self.model = model

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        nlp = spacy.load(self.model)
        return np.concatenate([nlp(" ".join(token.text for token in [token for token in nlp(doc) if
                                                                     not (token.pos_ == "PUNCT") and len(
                                                                         token) > 1])).vector.reshape(1, -1) for doc in X])

In [73]:
pipeline = Pipeline(
    # [("w2v", WordVectorTransformerPlus(model="fr_core_news_lg")),
    [('tfidf', TfidfVectorizer(stop_words="english")),
     ("clf", MLPClassifier(random_state=1, max_iter=300))]
)

In [18]:
df = get_data("data/dataset500.csv")

In [65]:
X = []
y = []
for comments in df["comments"]:
    for comment in comments:
        X.append(comment["comment"][0])
        y.append(comment["grade"][0])
        # if comment["grade"][0] >10:
        #     y.append(1)
        # else:
        #     y.append(0)

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True)

In [53]:
test = WordVectorTransformerPlus(model="fr_core_news_lg").fit_transform(X_train, y_train)

In [60]:
clf = LinearSVC()

In [56]:
test2 = WordVectorTransformerPlus(model="fr_core_news_lg").fit_transform(X_test, y_test)

In [12]:
test

<10955x40978 sparse matrix of type '<class 'numpy.float64'>'
	with 920006 stored elements in Compressed Sparse Row format>

In [61]:
clf.fit(test, y_train)

C:\DEV\WEM\projetWEM\venv\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [63]:
pred = clf.predict(test2)

In [74]:
res = pipeline.fit(X_train, y_train)

In [75]:
pred = pipeline.predict(X_test)

In [76]:
accuracy_score(y_test, pred)

0.2624446902654867